Packages importieren

In [37]:
import pandas as pd
import numpy as np
from numpy.linalg import svd
from scipy.optimize import least_squares
import plotly.graph_objects as go
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans

Funktionen importieren

In [38]:
def kabsch_algorithm(P, Q):
    p_centroid = np.mean(P, axis=0)
    q_centroid = np.mean(Q, axis=0)
    P_centered = P - p_centroid # Werte werden ins Zentrum des Koordinatensystems verschoben
    Q_centered = Q - q_centroid # Werte werden ins Zentrum des Koordinatensystems verschoben
    H = P_centered.T @ Q_centered # Korrelationsmatrix
    U, S, Vt = svd(H)
    V = Vt.T
    R = V @ U.T
    if np.linalg.det(R) < 0:
        V[:,-1] *= -1
        R = V @ U.T
    t = q_centroid - R @ p_centroid
    return R, t

def rotation_matrix_from_euler(alpha, beta, gamma):
    # Rotationsmatrizen für Eulerwinkel (Z-Y-X)
    Rz = np.array([[np.cos(alpha), -np.sin(alpha), 0],
                   [np.sin(alpha),  np.cos(alpha), 0],
                   [0,              0,             1]])
    Ry = np.array([[ np.cos(beta), 0, np.sin(beta)],
                   [0,             1, 0],
                   [-np.sin(beta), 0, np.cos(beta)]])
    Rx = np.array([[1,             0,              0],
                   [0, np.cos(gamma), -np.sin(gamma)],
                   [0, np.sin(gamma),  np.cos(gamma)]])
    return Rz @ Ry @ Rx

def residual(params, P, Q):
    alpha, beta, gamma, tx, ty, tz = params
    R_opt = rotation_matrix_from_euler(alpha, beta, gamma)
    t_opt = np.array([tx, ty, tz])
    P_transformed = (R_opt @ P.T).T + t_opt
    diff = P_transformed - Q
    return diff.flatten()


def gelenk0_TF(encoder_raw_position):
        """
        Transformation logic for Gelenk 0.
        """
        params = {
            "a": 2450.0517124242577,
            "b": 7867.202696571338,
            "c1": 5414.61226810207,
            "gamma0": -1.4348954481458154,
            "inverted": True,
                    }

        inverted = params['inverted']
        a, b, c1, gamma0 = params['a'], params['b'], params['c1'], params['gamma0']

        if (inverted == True):
            return (-1) * (np.arccos((a ** 2 + b ** 2 - (c1 + encoder_raw_position) ** 2) / (2 * a * b)) + gamma0)
        else:
            return np.arccos((a ** 2 + b ** 2 - (c1 + encoder_raw_position) ** 2) / (2 * a * b)) + gamma0
        
def gelenk1_TF(encoder_raw_position):
        """
        Transformation logic for Gelenk 1.
        """
        params={
        "offset": 33125,
        "resolution_bit": 18,
        "inverted": False}

        inverted = params['inverted']
        offset = params['offset']
        resolution_bit = params['resolution_bit']
        gain = pow(2, -resolution_bit) * 2 * np.pi

        adjusted_position = (encoder_raw_position - offset) % (2 ** resolution_bit)
        transformed_points_1 = []
        for position in adjusted_position:
            if position > (2 ** resolution_bit) / 2:
                value = position - (2 ** resolution_bit)
                transformed_points_1.append(value)
            else:
                 transformed_points_1.append(position)

        transformed_points_1 = pd.DataFrame(transformed_points_1, columns=['encoder_1'],index=encoder_raw_position.index)
        
        if (inverted == False):
            return transformed_points_1 * gain
        else:
            return (-1) * (transformed_points_1 * gain)

def gelenk2_TF(encoder_raw_position):
        """
        Transformation logic for Gelenk 2.
        """

        params ={
        "offset": 669,
        "gain": 0.0001, 
        "inverted": False}

        inverted = params['inverted']
        offset = params['offset']
        gain = params['gain']
    
        if (inverted == False):
            return (encoder_raw_position - offset) * gain
        else:
            return (-1) * ((encoder_raw_position - offset) * gain)

def gelenk3_TF(encoder_raw_position):
        """
        Transformation logic for Gelenk 3.
        """

        params={
        "offset": 150339,
        "resolution_bit": 18,
        "inverted": False}

        inverted = params['inverted']
        offset = params['offset']
        resolution_bit = params['resolution_bit']
        gain = pow(2, -resolution_bit) * 2 * np.pi

        adjusted_position = (encoder_raw_position - offset) % (2 ** resolution_bit)
        transformed_points_3 = []
        for position in adjusted_position:
            if position > (2 ** resolution_bit) / 2:
                value = position - (2 ** resolution_bit)
                transformed_points_3.append(value)
            else:
                 transformed_points_3.append(position)

        transformed_points_3 = pd.DataFrame(transformed_points_3, columns=['encoder_3'],index=encoder_raw_position.index)
        
        if (inverted == False):
            return transformed_points_3 * gain
        else:
            return (-1) * (transformed_points_3 * gain)
        

def gelenk4_TF(encoder_raw_position):
        """
        Transformation logic for Gelenk 4.
        """

        params={
        "offset": 186926,
        "resolution_bit": 18,
        "inverted": True}

        inverted = params['inverted']
        offset = params['offset']
        resolution_bit = params['resolution_bit']
        gain = pow(2, -resolution_bit) * 2 * np.pi

        adjusted_position = (encoder_raw_position - offset) % (2 ** resolution_bit)
        transformed_points_4 = []
        for position in adjusted_position:
            if position > (2 ** resolution_bit) / 2:
                value = position - (2 ** resolution_bit)
                transformed_points_4.append(value)
            else:
                 transformed_points_4.append(position)

        transformed_points_4 = pd.DataFrame(transformed_points_4, columns=['encoder_4'],index=encoder_raw_position.index)
        
        if (inverted == False):
            return transformed_points_4 * gain
        else:
            return (-1) * (transformed_points_4 * gain)

def gelenk5_TF(encoder_raw_position):
        """
        Transformation logic for Gelenk 5.
        """

        params={
        "offset": 33459832,
        "gain": 0.0000343397756528412,
        "inverted": False}

        inverted = params['inverted']
        offset = params['offset']
        gain = params['gain']

        if (inverted == False):
            return (encoder_raw_position - offset) * gain
        else:
            return (-1) * ((encoder_raw_position - offset) * gain)

def gelenk6_TF(encoder_raw_position):
        """
        Transformation logic for Gelenk 6.
        """

        params={
        "offset": 1244,
        "resolution_bit": 13,
        "inverted": True}

        inverted = params['inverted']
        offset = params['offset']
        resolution_bit = params['resolution_bit']
        gain = pow(2, -resolution_bit) * 2 * np.pi

        adjusted_position = (encoder_raw_position - offset) % (2 ** resolution_bit)
        transformed_points_6 = []
        for position in adjusted_position:
            if position > (2 ** resolution_bit) / 2:
                value = position - (2 ** resolution_bit)
                transformed_points_6.append(value)
            else:
                 transformed_points_6.append(position)

        transformed_points_6 = pd.DataFrame(transformed_points_6, columns=['encoder_6'],index=encoder_raw_position.index)
        
        if (inverted == False):
            return transformed_points_6 * gain
        else:
            return (-1) * (transformed_points_6 * gain)

def gelenk7_TF(encoder_raw_position):
        """
        Transformation logic for Gelenk 7.
        """

        params={
        "offset": 7266,
        "resolution_bit": 13,
        "inverted": False}

        inverted = params['inverted']
        offset = params['offset']
        resolution_bit = params['resolution_bit']
        gain = pow(2, -resolution_bit) * 2 * np.pi

        adjusted_position = (encoder_raw_position - offset) % (2 ** resolution_bit)
        transformed_points_7 = []
        for position in adjusted_position:
            if position > (2 ** resolution_bit) / 2:
                value = position - (2 ** resolution_bit)
                transformed_points_7.append(value)
            else:
                 transformed_points_7.append(position)

        transformed_points_7 = pd.DataFrame(transformed_points_7, columns=['encoder_3'],index=encoder_raw_position.index)
        
        if (inverted == False):
            return transformed_points_7 * gain
        else:
            return (-1) * (transformed_points_7 * gain)

Messdaten aller Messtage 21.01., 01.02., 10.02., 03.03. (Position 1) und 03.03. (Position 2) importieren

In [39]:
# Daten vom 21.01.2025 importieren und strukturieren
data_ts_21_01 = pd.read_csv('../data/Rohdaten_TS_21_01_25.csv', delimiter=';', index_col=0).sort_index()
data_encoder_21_01 = pd.read_csv('../data/Encoderwerte_21_01_25.csv', delimiter=';', index_col=0).sort_index()
validation_data_TS_21_01 = pd.read_csv('../data/Rohdaten_TS_21_01_V_25.csv', delimiter=';', index_col=0).sort_index()
validation_data_encoder_21_01 = pd.read_csv('../data/Encoderwerte_21_01_V_25.csv', delimiter=';', index_col=0).sort_index()

encoder_points_21_01 = (data_encoder_21_01[['x_Encoder', 'y_Encoder', 'z_Encoder']]/1000)
ts_points_21_01 = data_ts_21_01[['x_TS', 'y_TS', 'z_TS']]

encoder_points_validation_21_01 = validation_data_encoder_21_01[['x_Encoder', 'y_Encoder', 'z_Encoder']].values/1000
ts_points_validation_21_01 = validation_data_TS_21_01[['x_TS', 'y_TS', 'z_TS']].values 

# Daten vom 01.02.2025 importieren und strukturieren
data = pd.read_csv('../data/Modellentwicklung_Encoderwerte.csv', sep=';', index_col=0)
raw_data = pd.read_csv('../data/Modellentwicklung_Rohdaten_Totalstation.csv', delimiter=';', index_col=0)

raw_data.columns = ['x_TS', 'y_TS', 'z_TS']
data_positioning = data[
    (data['Norm'] < 20) & 
    (data['x_Encoder'] < 2000) & 
    (data['x_Encoder'] > -1200) & 
    (data['y_Encoder'] < 8000) & 
    (data['z_Encoder'] > -1000) & 
    (data['z_Encoder'] < 2200)
]
data_positioning.index = data_positioning.index
raw_data_positioning = raw_data[raw_data.index.isin(data_positioning.index)]
ts_points_01_02 = raw_data_positioning[['x_TS','y_TS','z_TS']].sort_index()
encoder_points_01_02 = data_positioning[['x_Encoder','y_Encoder','z_Encoder']].sort_index()/1000
data_encoder_01_02 = data_positioning[['x_Encoder', 'y_Encoder', 'z_Encoder','encoder_0','encoder_1','encoder_2','encoder_3','encoder_4','encoder_5','encoder_6','encoder_7']].sort_index()
used_indices = set(data_positioning.index)
available_indices = list(set(data.index) - used_indices)
selected_indices = np.random.choice(available_indices, 20, replace=False)
validation_data_encoder_01_02 = data.loc[selected_indices].sort_index()
validation_data_TS_01_02 = raw_data.loc[selected_indices].sort_index()
encoder_points_validation_01_02 = validation_data_encoder_01_02[['x_Encoder', 'y_Encoder', 'z_Encoder']].values/1000 
ts_points_validation_01_02 = validation_data_TS_01_02[['x_TS', 'y_TS', 'z_TS']].values

# Daten vom 10.02.2025 importieren und strukturieren
data = pd.read_csv('../data/Encoderwerte_Rohdaten_10_02_25.csv', delimiter=';',index_col=0)
ts_points_10_02 = data[['x_TS', 'y_TS', 'z_TS']].sort_index()
encoder_points_10_02 = data[['x_Encoder', 'y_Encoder','z_Encoder']].sort_index()
data_encoder_10_02 = data[['x_Encoder', 'y_Encoder','z_Encoder','encoder_0','encoder_1','encoder_2','encoder_3','encoder_4','encoder_5','encoder_6','encoder_7']].sort_index()

# Daten vom 25.02.2025 importieren und strukturieren
data_TS_25_02 = pd.read_csv('../data/Rohdaten_TS_25_02_E_25.csv', delimiter=';', index_col=0).sort_index()
data_encoder_25_02 = pd.read_csv('../data/Encoderwerte_25_02_E_25.csv', delimiter=';', index_col=0).sort_index()
ts_points_25_02 = data_TS_25_02[['x_TS', 'y_TS', 'z_TS']]
encoder_points_25_02 = data_encoder_25_02[['x_Encoder', 'y_Encoder','z_Encoder']]/1000
validation_data_encoder_25_02 = pd.read_csv('../data/Encoderwerte_25_02_25.csv', delimiter=';', index_col=0).sort_index()
validation_data_TS_25_02 = pd.read_csv('../data/Rohdaten_TS_25_02_25.csv', delimiter=';', index_col=0).sort_index()
encoder_points_validation_25_02 = validation_data_encoder_25_02[['x_Encoder', 'y_Encoder', 'z_Encoder']].values/1000
ts_points_validation_25_02 = validation_data_TS_25_02[['x_TS', 'y_TS', 'z_TS']].values

# Daten vom 03.03.2025 (Position 1) importieren und strukturieren
data_TS_03_03 = pd.read_csv('../data/Rohdaten_TS_03_03_25_Position_1_Einmessung.csv', delimiter=';', index_col=0).sort_index()
data_encoder_03_03 = pd.read_csv('../data/Encoderwerte_03_03_25_Position_1_Einmessung.csv', delimiter=';', index_col=0).sort_index()
ts_points_03_03 = data_TS_03_03[['x_TS', 'y_TS', 'z_TS']]
encoder_points_03_03 = data_encoder_03_03[['x_Encoder', 'y_Encoder','z_Encoder']]/1000
validation_data_encoder_03_03 = pd.read_csv('../data/Encoderwerte_03_03_25_Position_1.csv', delimiter=';', index_col=0).sort_index()
validation_data_TS_03_03 = pd.read_csv('../data/Rohdaten_TS_03_03_25_Position_1.csv', delimiter=';', index_col=0).sort_index()
validation_data_encoder_03_03 = validation_data_encoder_03_03[~validation_data_encoder_03_03.index.str.endswith(("second_stage_corrected", "first_stage_corrected"))].sort_index()
encoder_points_validation_03_03 = validation_data_encoder_03_03[['x_Encoder', 'y_Encoder', 'z_Encoder']].values/1000
ts_points_validation_03_03 = validation_data_TS_03_03[['x_TS', 'y_TS', 'z_TS']].values

# Daten vom 03.03.2025 (Position 2) importieren und strukturieren
data_TS_03_03_2 = pd.read_csv('../data/Rohdaten_TS_03_03_25_Position_2_Einmessung.csv', delimiter=';', index_col=0).sort_index()
data_encoder_03_03_2 = pd.read_csv('../data/Encoderwerte_03_03_25_Position_2_Einmessung.csv', delimiter=';', index_col=0).sort_index()
ts_points_03_03_2 = data_TS_03_03_2[['x_TS', 'y_TS', 'z_TS']]
encoder_points_03_03_2 = data_encoder_03_03_2[['x_Encoder', 'y_Encoder','z_Encoder']]/1000

validation_data_encoder_03_03_2 = pd.read_csv('../data/Encoderwerte_03_03_25_Position_2.csv', delimiter=';', index_col=0).sort_index()
validation_data_TS_03_03_2 = pd.read_csv('../data/Rohdaten_TS_03_03_25_Position_2.csv', delimiter=';', index_col=0).sort_index()
validation_data_encoder_03_03_2 = validation_data_encoder_03_03_2[~validation_data_encoder_03_03_2.index.str.endswith(("second_stage_corrected", "first_stage_corrected"))].sort_index()
encoder_points_validation_03_03_2 = validation_data_encoder_03_03_2[['x_Encoder', 'y_Encoder', 'z_Encoder']].values/1000
ts_points_validation_03_03_2 = validation_data_TS_03_03_2[['x_TS', 'y_TS', 'z_TS']].values

# Daten zusammenfassen
datasets = {
    '21_01': (encoder_points_21_01.values, ts_points_21_01.values),
    '01_02': (encoder_points_01_02.values, ts_points_01_02.values),
    '10_02': (encoder_points_10_02.values, ts_points_10_02.values), 
    '25_02': (encoder_points_25_02.values, ts_points_25_02.values),
    '03_03_1': (encoder_points_03_03.values, ts_points_03_03.values),
    '03_03_2': (encoder_points_03_03_2.values, ts_points_03_03_2.values),
}

Transformationen und 3D-Vektorplot für jedes Datenset bestimmen

In [40]:
deltas_total = pd.DataFrame()

# For-Loop für die Transformation
for day, (P, Q) in datasets.items():
    # Schritt 1: Kabsch
    R_est, t_est = kabsch_algorithm(P, Q)
    
    # Schritt 2: Least Squares Optimierung
    x0 = np.array([0.0, 0.0, 0.0, t_est[0], t_est[1], t_est[2]])
    result = least_squares(residual, x0, args=(P, Q), loss='huber', f_scale=1.0)
    alpha_opt, beta_opt, gamma_opt, tx_opt, ty_opt, tz_opt = result.x
    R_opt = rotation_matrix_from_euler(alpha_opt, beta_opt, gamma_opt)
    t_opt = np.array([tx_opt, ty_opt, tz_opt])

    # Schritt 3: Transformation der Encoderpunkte
    P_transformed = (R_opt @ P.T).T + t_opt

    # Schritt 4: Fehler (Vektoren)
    deltas = (Q - P_transformed) * 1000  # Fehler in mm

    if day != '10_02':
        deltas_total = pd.concat([deltas_total, pd.DataFrame(deltas, columns=['x', 'y', 'z'])], axis=0)

Zusammenfassung aller Encoderwerte und Abweichungen

In [41]:
# Encoderwerte und Deltas in jeweils einem DataFrame zusammenfassen
encoder_total = pd.concat([data_encoder_21_01, data_encoder_01_02, data_encoder_25_02, data_encoder_03_03, data_encoder_03_03_2])
deltas_total.index = encoder_total.index
encoder_total_1 = pd.concat([encoder_total, deltas_total], axis=1)
encoder_total_1.columns = ['x_Encoder', 'y_Encoder', 'z_Encoder','encoder_0','encoder_1','encoder_2','encoder_3','encoder_4','encoder_5','encoder_6','encoder_7', 'delta_x', 'delta_y', 'delta_z']
encoder_total_1['norm'] = np.linalg.norm(encoder_total_1[['delta_x', 'delta_y', 'delta_z']], axis=1)

Cluster erstellen um Messpunkte zu filtern und auszuwählen

In [42]:
# 8 Clusterzentren definieren für KMeans
centroids = np.array([
    [-750, 7000, -250],   # Bottom-Back-Left
    [750, 7000, -250],    # Bottom-Back-Right
    [-750, 8000, -250],   # Bottom-Front-Left
    [750, 8000, -250],    # Bottom-Front-Right
    [-750, 7000, 1250],   # Top-Back-Left
    [750, 7000, 1250],    # Top-Back-Right
    [-750, 8000, 1250],   # Top-Front-Left
    [750, 8000, 1250]     # Top-Front-Right
])

# Datenpunkte für KMeans
data = encoder_total_1[['x_Encoder', 'y_Encoder', 'z_Encoder']].values

# Nächstgelegenes Clusterzentrum für jeden Punkt finden
neighbors = NearestNeighbors(n_neighbors=1)
neighbors.fit(centroids)
distances, indices = neighbors.kneighbors(data)

# DataFrame erstellen mit den Datenpunkten und Clusterzugehörigkeit
df_data = pd.DataFrame(data, columns=['x', 'y', 'z'], index=encoder_total_1.index)
df_data['cluster'] = indices.flatten()

encoder_total_1['cluster'] = df_data['cluster']

# Nur Werte mit Norm < 12 mm behalten
encoder_total_1 = encoder_total_1[encoder_total_1['norm'] < 12]

3D-Vektorplot der geclusterten Einmesspunkte

In [43]:
# Farben für jedes Cluster
cluster_colors = {
    0: "rgb(230, 25, 75)",    # Rot
    1: "rgb(0, 130, 200)",    # Blau
    2: "rgb(60, 180, 75)",    # Grün
    3: "rgb(255, 140, 0)",    # Orange
    4: "rgb(145, 30, 180)",   # Lila
    5: "rgb(0, 190, 255)",    # Hellblau
    6: "rgb(255, 225, 25)",   # Gelb
    7: "rgb(245, 50, 150)"    # Pink
}

# Positionen der Colorbars
colorbar_positions = {
    0: (0.85, 1),
    1: (0.85, 0.75),
    2: (0.85, 0.5),
    3: (0.85, 0.25),
    4: (0.95, 1),
    5: (0.95, 0.75),
    6: (0.95, 0.5),
    7: (0.95, 0.25)
}

# Plot anlegen
fig = go.Figure()
sizeref_factor = 10  # Skalieren der Pfeilgrößen

# For-Loop für die Erstellung der Datenpunkte des 3D-Plots
for cluster_id in sorted(encoder_total_1['cluster'].unique()):
    cluster_df = encoder_total_1[encoder_total_1['cluster'] == cluster_id]
    color = cluster_colors.get(cluster_id, "rgb(150, 150, 150)") 
    x_bar, y_bar = colorbar_positions.get(cluster_id, (1.0, 0.5))

    quiver = go.Cone(
        x=cluster_df["x_Encoder"],
        y=cluster_df["y_Encoder"],
        z=cluster_df["z_Encoder"],
        u=cluster_df["delta_x"],
        v=cluster_df["delta_y"],
        w=cluster_df["delta_z"],
        opacity=1,
        sizemode="raw",
        sizeref=sizeref_factor,
        colorscale=[[0, color], [1, color]],
        showscale=True,
        colorbar=dict(
            title=f"Cluster {cluster_id}",
            len=0.25,
            x=x_bar,
            y=y_bar
        ),
        name=f"Cluster {cluster_id}"
    )

    fig.add_trace(quiver)

# Layout anpassen
fig.update_layout(
    title='3D Vektorplot der geclusterten Einmesspunkte',
    scene=dict(
        xaxis_title="<i>x</i> in mm",
        yaxis_title="<i>y</i> in mm",
        zaxis_title="<i>z</i> in mm"
    )
)

# Plot speichern
fig.write_html("../results/Diagramme/Einmessverfahren/3D_Vektorplot_Cluster_Einmesspunkte.html")


Auswahl 5 zufälliger Werte aus jedem Cluster zur Implementierung des Einmessverfahrens in die Maschinensteuerung

In [44]:
# Anzahl der zu samplenden Punkte pro Cluster
n_samples = 5

# Leere Liste für die gesampelten Punkte
sampled_points = []

# For-Loop für die Auswahl der Punkte aus jedem Cluster
for cluster_id in range(8):
    # Datenpunkte des Clusters auswählen
    cluster_data = encoder_total_1[encoder_total_1['cluster'] == cluster_id]
    
    # Zufällige Auswahl der Punkte
    sampled_cluster = cluster_data.sample(n=min(n_samples, len(cluster_data)), random_state=42)
    
    # An Liste anhängen
    sampled_points.append(sampled_cluster)

# Zusammenführen der gesampelten Punkte in einen DataFrame
df_sampled = pd.concat(sampled_points)

Plotten der gefilterten Punkte

In [45]:
sizeref_factor = 10  # Adjust this factor for desired visual scaling

# Create vectors (quivers) for errors before transformation
quivers_sampled_points = go.Cone(
    x=df_sampled["x_Encoder"],
    y=df_sampled["y_Encoder"],
    z=df_sampled["z_Encoder"],
    u=df_sampled['delta_x'],
    v=df_sampled['delta_y'],
    w=df_sampled['delta_z'],
    opacity= 1,
    sizemode="raw",
    sizeref=sizeref_factor,  # Use calculated sizeref for errors before
    colorscale='Reds',
    colorbar=dict(
        title="Abweichung in mm",
        len=0.5,  # Length of the colorbar
        x=0.85,   # Position on the x-axis
        y=0.7   # Position on the y-axis
    ),
    name='Einmesspunkte'
)

# Create the figure and add traces
fig = go.Figure(data=[quivers_sampled_points])

# Set plot layout
fig.update_layout(
    title='3D Vektorplot der finalen Einmesspunkte',
    scene=dict(
        xaxis_title="<i>x</i> in mm",
        yaxis_title='<i>y</i> in mm',
        zaxis_title='<i>z</i> in mm'
    )
)

# Save the plot to an HTML file
fig.write_html("../results/Diagramme/Einmessverfahren/3D_Vektorplot_Zusammenfassung_Einmesspunkte.html")

Umrechnung der Winkel in Radianten

In [46]:
positioning_points = pd.DataFrame(columns=['x_Encoder', 'y_Encoder', 'z_Encoder', 'encoder_0', 'encoder_1', 'encoder_2', 'encoder_3', 'encoder_4', 'encoder_5', 'encoder_6', 'encoder_7'],index=df_sampled.index)
positioning_points[['x_Encoder', 'y_Encoder', 'z_Encoder','cluster']] = df_sampled[['x_Encoder', 'y_Encoder', 'z_Encoder','cluster']]
positioning_points['encoder_0'] = gelenk0_TF(df_sampled['encoder_0'])
positioning_points['encoder_1'] = gelenk1_TF(df_sampled['encoder_1'])
positioning_points['encoder_2'] = gelenk2_TF(df_sampled['encoder_2'])
positioning_points['encoder_3'] = gelenk3_TF(df_sampled['encoder_3'])
positioning_points['encoder_4'] = gelenk4_TF(df_sampled['encoder_4'])
positioning_points['encoder_5'] = gelenk5_TF(df_sampled['encoder_5'])
positioning_points['encoder_6'] = gelenk6_TF(df_sampled['encoder_6'])
positioning_points['encoder_7'] = gelenk7_TF(df_sampled['encoder_7'])
positioning_points.index.name = 'ID'
positioning_points.to_csv('../results/Tabellen/Einmesspunkte_Encoderstellungen.csv', index=True)

C:\Users\si159\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

invalid value encountered in arccos

